## 文本分类

模型 预测

常用模型
    DAN : 情感分类
    双向RNN:
    堆叠神经网络
    CNN:

使用PyTorch模型和TorchText做情感分析
检测一段文字的情感是证明还是负面的。数据集：电影评论IMDb数据集

模型从简单到复杂：
   * Word Averaging 模型
   * RNN / LSTM 模型
   * CNN 模型

### 准备数据
* TorchText中的Field，决定数据怎么被处理
* 使用Text Field来处理定义评论，使用Label field处理2中情感类型pos， neg
* Text Field中tokenize=‘spacy' 表示我们用SpaCy tokenizer来tokenize英文语句。如果不使用该参数，默认使用空格来分词
* 安装SpaCy   `pip install spacy & python -m spacy donwload en
* label 有LabelField定义

In [1]:
import torch
from torch.utils.data import DataLoader
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext import vocab

SEED = 1
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('torch:', torch.__version__)

torch: 1.9.0+cpu


In [2]:
from torchtext.datasets import IMDB

train_iter, test_iter = IMDB()
print(f' len(train_iter): {len(train_iter)}, test: {len(test_iter)}')

 len(train_iter): 25000, test: 25000


In [3]:
# 使用 build_vocab_from_iterator 预处理
tokenizer = get_tokenizer('basic_english')
from torchtext.vocab import build_vocab_from_iterator
train_iter = IMDB(split='train')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
VOCAB_SIZE = len(vocab)
print(f'VOCAB_SIZE: {VOCAB_SIZE}')

VOCAB_SIZE: 100683


In [9]:
# Prepare data processing pipelines
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x == 'neg' else 1

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [10]:
#  划分训练，验证集，
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

total_accu = None
train_iter, test_iter = IMDB()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.8)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=True, collate_fn=collate_batch)

In [11]:
print(next(train_dataloader._get_iterator()))

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1]), tensor([ 12, 298,   8,  ...,   2,   2,   2]), tensor([    0,   246,   702,  1650,  1688,  1998,  2234,  2513,  2869,  3020,
         3199,  3312,  3737,  4311,  4635,  4779,  5108,  5263,  5570,  5741,
         5888,  5961,  6103,  6288,  6722,  6998,  7096,  7282,  7508,  7613,
         7771,  8050,  8398,  8534,  8654,  9122,  9266,  9400,  9715,  9798,
        10072, 10362, 10513, 10639, 10811, 11564, 11779, 11940, 12156, 12547,
        12723, 13430, 13562, 13695, 13859, 14018, 14277, 14892, 15124, 15326,
        15570, 15880, 16031, 16163]))


###  Word Averaging模型

1. 定义模型:  nn.EmbeddingBag layer plus a linear layer for the classification purpose.
2. 关于实现：avg_pool2d做average pooling。（sent_len, embedding_dim) -> (1,embedding_dim)
3. note:avg_pool2d的kernel_size是（embedded.shape[1],1），所以句子长度这个维度会被压扁

In [22]:
from torch import nn
import torch.nn.functional as F
#模型构建
class WordAVGModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(WordAVGModel,self).__init__()
        # TODO 此处有问题
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # 单词变成词向量
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text) # [sent len, batch size, emb dim]
        print('shape of embedded', embedded.shape)
        # TODO 此处有问题
        embedded = embedded.permute(1, 0, 2) # [batch size, sent len, emb dim]
        # 逐列求平均
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) # [batch size, embedding_dim]
        return self.fc(pooled)


In [23]:
EMBEDDING_SIZE = 64
OUTPUT_DIM = 1
model = WordAVGModel(VOCAB_SIZE, EMBEDDING_SIZE, 1).to(device)


In [12]:
print(vocab.vectors)

AttributeError: 'Vocab' object has no attribute 'vectors'

In [ ]:
import torchtext
vector = torchtext.vocab.pretrained_aliases['glove.6B.100d']
model.embedding.weight.data.copy_(vector())

In [24]:
UNK_IDX = vocab.get_stoi()["<unk>"]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

In [25]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
print(f'OUTPUT_DIM:{OUTPUT_DIM}, VOCAB_SIZE:{VOCAB_SIZE}, EMBEDDING_SIZE:{EMBEDDING_SIZE}')


OUTPUT_DIM:1, VOCAB_SIZE:100683, EMBEDDING_SIZE:64


In [26]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [27]:
# 启动训练
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    # if total_accu is not None and total_accu > accu_val:
    #   scheduler.step()
    # else:
    total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,time.time() - epoch_start_time,accu_val))


shape of embedded torch.Size([18966, 64])


RuntimeError: number of dims don't match in permute

In [ ]:
# 启动测试
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

## RNN模型

- 下面我们尝试把模型换成一个**recurrent neural network** (RNN)。
RNN经常会被用来encode一个sequence

     $$h_t = \text{RNN}(x_t, h_{t-1})$$

- 我们使用最后一个hidden state $h_T$来表示整个句子。

- 然后我们把$h_T$通过一个线性变换$f$，然后用来预测句子的情感。

![](assets/sentiment1.png)

![](assets/sentiment7.png)


In [31]:
from torch import nn
import torch.nn.functional as F
#模型构建
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, hidden_size, dropout):
        super(RNNModel,self).__init__()
        # TODO 此处有问题
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # 单词变成词向量
        self.lstm = nn.LSTM(embedding_dim, hidden_size)
        self.linear = nn.Linear(hidden_size, output_dim)
        self.dropout = nn.Dropout(dropout) # 每个位置

    def forward(self, text):
        embedded = self.embedding(text) # [sent len, batch size, emb dim]

        embedded = self.dropout(embedded)
        print('shape of embedded', embedded.shape) # TODO 此处有问题
        output, (hidden, cell) = self.lstm(embedded) # input must have 3 dimensions, got 2
        # hidden: 1 * batch_sze * hidden_size
        hidden = self.dropout(hidden.squeenze())
        return self.linear(hidden)

## CNN模型

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters,
                 filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, out_channels = n_filters,
                                              kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        text = text.permute(1, 0) # [batch size, sent len]
        embedded = self.embedding(text) # [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1) # [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        #pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]

        return self.fc(cat)
